In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import evaluate
from transformers import AutoTokenizer

/home/leroy/.pyenv/versions/3.12.3/envs/data-analysis/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset1 = pd.read_csv('datasets/troll_dataset.csv')
dataset2 = pd.read_csv('datasets/metootweets_cleaned.csv')
dataset3 = pd.read_csv('datasets/WHtweets_cleaned.csv')

In [3]:
dataset1.drop(columns = ['external_author_id', 'author', 'region', 'language', 'publish_date', 'harvested_date', 'following', 'followers', 'updates', 'account_type', 'new_june_2018', 'retweet', 'account_category', 'has_emoji'], inplace=True)
dataset2.drop(columns = ['id', 'insertdate', 'twitterhandle', 'followers', 'hashtagsearched', 'tweetid', 'dateoftweet'], inplace=True)
dataset3.drop(columns = ['tweet_id'], inplace=True)

dataset1['label'] = 1
dataset2['label'] = 0

In [4]:
dataset1.rename(columns = {'content':'tweet'}, inplace=True)
dataset2.rename(columns = {'text':'tweet'}, inplace=True)
dataset3.rename(columns = {'text':'tweet'}, inplace=True)

In [5]:
print (dataset1.__len__(), dataset2.__len__() +  dataset3.__len__())

# dropping rows from dataset1 to get the same number of rows as dataset2 and dataset3
dataset1 = dataset1.sample(n=dataset2.__len__() +  dataset3.__len__(), random_state=1)

2128608 361998


In [6]:
pd.concat([dataset1, dataset2, dataset3]).to_csv('datasets/class_dataset.csv', index=False)
dataset = pd.read_csv('datasets/class_dataset.csv')

print(dataset.__len__())

723996


In [7]:
X = dataset['tweet']
y = dataset['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [8]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

accuracy = evaluate.load('accuracy')